# Imports

In [1]:
import os
import sys
import json
import glob
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', 180)
pd.set_option('display.max_rows', 200)

In [3]:
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../../eventpredictor'))

import eventpredictor.config as paths
from eventpredictor.data.download_data import FootballScraper as fs

2024-06-02 13:56:37.366 | INFO     | config:<module>:11 - PROJ_ROOT path is: /Users/maurycystopowski/Documents/Stuff/Projects/eventpredictor
2024-06-02 13:56:37.381 | INFO     | eventpredictor.config:<module>:11 - PROJ_ROOT path is: /Users/maurycystopowski/Documents/Stuff/Projects/eventpredictor


# Loading data

In [4]:
events_file_name = os.path.join(paths.SCHEDULED_EVENTS, 'scheduled_events.csv')
odds_file_name = os.path.join(paths.ODDS, 'odds.csv')

In [5]:
cols_to_select = [
    'id',
    'startTimestamp',
    'tournament_slug',
    'tournament_category_slug',
    'season_name',
    'status_description',
    'status_type',
    'winnerCode',
    'homeTeam_slug',
    'awayTeam_slug',
    'homeScore_period1',
    'homeScore_period2',
    'homeScore_normaltime',
    'awayScore_period1',
    'awayScore_period2',
    'awayScore_normaltime',
]

In [6]:
events_df = pd.read_csv(events_file_name, usecols=cols_to_select)
events_df = events_df.drop_duplicates()

In [7]:
odds_df = pd.read_csv(odds_file_name)
odds_df = odds_df.drop_duplicates()

# Merge and save to csv

In [8]:
df = events_df.merge(odds_df, on='id', how='left')
df = df.sort_values('startTimestamp')
df = df.reset_index(drop=True)

In [9]:
df.shape

(482602, 19)

In [10]:
df.head()

,winnerCode,id,startTimestamp,tournament_slug,tournament_category_slug,season_name,status_description,status_type,homeTeam_slug,awayTeam_slug,homeScore_period1,homeScore_period2,homeScore_normaltime,awayScore_period1,awayScore_period2,awayScore_normaltime,1,2,X
0,1.0,5714392,1432987200,nb-i,hungary,NB I 14/15,Ended,finished,ujpest,kecskemet-te,2.0,1.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN
1,1.0,6750849,1432987200,2nd-division-avd-4,norway,"2nd Division, avd. 4 2015",Ended,finished,floro,rodde,2.0,2.0,4.0,0.0,0.0,0.0,NaN,NaN,NaN
2,2.0,6689991,1432987200,2nd-division-avd-3,norway,"2nd Division, avd. 3 2015",Ended,finished,fana,flekkeroy,1.0,0.0,1.0,0.0,2.0,2.0,NaN,NaN,NaN
3,2.0,5617268,1432987200,1-liga,russia,Football National League 14/15,Ended,finished,fc-tyumen,volgar-astrakhan,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN
4,2.0,5714924,1432989000,regionalliga-west,austria-amateur,Regionalliga West 14/15,Ended,finished,sc-rheindorf-altach-ii,wsg-tirol,0.0,1.0,1.0,2.0,4.0,6.0,NaN,NaN,NaN


In [12]:
df.to_csv(os.path.join(paths.INTERIM_DATA_DIR, 'events_with_odds_merged.csv'), index=False)

In [13]:
df['startTimestamp'] = pd.to_datetime(df['startTimestamp'], unit='s')

In [14]:
df.sample(10).T

,352174,422636,209774,73010,273219,258753,181675,296422,428547,182104
winnerCode,2.0,1.0,1.0,2.0,2.0,3.0,3.0,2.0,3.0,1.0
id,9576394,10913571,8231022,7105666,9007934,8658107,8089386,8738668,10388638,8102709
startTimestamp,2021-10-16 14:00:00,2023-02-04 12:00:00,2019-07-31 00:30:00,2016-11-05 12:00:00,2020-09-16 16:00:00,2020-06-10 15:00:00,2019-01-12 23:00:00,2020-12-23 20:00:00,2023-03-12 12:30:00,2019-01-19 10:00:00
tournament_slug,premier-league,thai-league-1,conmebol-sudamericana,1-liga,club-friendly-games,1st-division,liga-mx-clausura,ligue-1,2-bundesliga,club-friendly-games
tournament_category_slug,england,thailand,south-america,russia,world,denmark,mexico,france,germany,world
season_name,Premier League 21/22,Thai League 1 22/23,Copa Sudamericana 2019,Football National League 16/17,Club Friendly Games 2020,1st Division 19/20,Primera Division 18/19. Clausura,Ligue 1 20/21,2nd Bundesliga 22/23,Club Friendly Games 2019
status_description,Ended,Ended,Ended,Ended,Ended,Ended,Ended,Ended,Ended,Ended
status_type,finished,finished,finished,finished,finished,finished,finished,finished,finished,finished
homeTeam_slug,aston-villa,sukhothai,fluminense,mordovia-saransk,cd-mosconia,hb-koge,monterrey,montpellier,hannover-96,as-trencin
awayTeam_slug,wolverhampton,muang-thong-united,penarol,spartak-moscow-ii,real-oviedo-vetusta,fc-fredericia,club-leon,lille,fc-hansa-rostock,slezsky-fc-opava


# Tournaments with more than 2000 rows

In [16]:
countries_to_remove = {
    'africa',
    'asia',
    'esoccer',
    'europe',
    'international',
    'north-central-america',
    'oceania',
    'simulated-reality-women',
    'south-africa',
    'south-america',
    'world'
}

In [17]:
selected_tournaments = df[~df['tournament_category_slug'].isin(countries_to_remove)][['tournament_category_slug', 'tournament_slug', 'id']]\
    .groupby(['tournament_category_slug', 'tournament_slug'], as_index=False)['id'].count()
selected_tournaments = selected_tournaments[selected_tournaments['id'] > 2000].reset_index(drop=True).rename(columns={'id': 'count'})

In [18]:
selected_tournaments

,tournament_category_slug,tournament_slug,count
0,algeria,ligue-1,2436
1,argentina,liga-profesional,3189
2,argentina,primera-b-nacional,2397
3,belarus,vysshaya-league,2091
4,belgium,pro-league,2429
5,brazil,brasileirao-serie-a,3424
6,brazil,brasileirao-serie-b,3422
7,czech-republic,1-liga,2225
8,czech-republic,fnl,2077
9,egypt,premier-league,2791


In [19]:
selected_tournaments_set = (selected_tournaments['tournament_category_slug'] + selected_tournaments['tournament_slug']).values

In [20]:
df = df.loc[(df['tournament_category_slug'] + df['tournament_slug']).isin(selected_tournaments_set)]
df = df.sort_values('startTimestamp').reset_index(drop=True)
df = df.reset_index(drop=True)

In [21]:
df.sample(5)

,winnerCode,id,startTimestamp,tournament_slug,tournament_category_slug,season_name,status_description,status_type,homeTeam_slug,awayTeam_slug,homeScore_period1,homeScore_period2,homeScore_normaltime,awayScore_period1,awayScore_period2,awayScore_normaltime,1,2,X
141746,1.0,9645722,2022-02-20 11:30:00,serie-a,italy,Serie A 21/22,Ended,finished,fiorentina,atalanta,0.0,1.0,1.0,0.0,0.0,0.0,31/20,163/100,5/2
169828,3.0,10388709,2023-05-07 11:30:00,2-bundesliga,germany,2nd Bundesliga 22/23,Ended,finished,1-fc-heidenheim,1-fc-magdeburg,0.0,0.0,0.0,0.0,0.0,0.0,11/20,19/4,10/3
120902,2.0,9238655,2021-03-14 17:00:00,stoiximan-super-league,greece,Super League 1 20/21,Ended,finished,ae-larisa,olympiacos,0.0,NaN,1.0,1.0,NaN,3.0,9/1,33/100,7/2
31689,1.0,7127106,2016-12-03 15:00:00,national-league,england,National League 16/17,Ended,finished,north-ferriby-united,gateshead,1.0,0.0,1.0,0.0,0.0,0.0,6/1,9/20,10/3
113901,3.0,9196926,2020-12-10 12:00:00,1st-division,norway,1st Division 2020,Ended,finished,strommen-if,stjordalsblink,1.0,NaN,1.0,1.0,NaN,1.0,83/100,9/4,3/1


In [22]:
df.shape

(187130, 19)

# status_description filtering

In [23]:
status_description_set = {
    # 'Abandoned',
    'Ended', 
    # 'Retired', 
    # 'Walkover', 
}

In [24]:
df['status_description'].value_counts()

status_description
Ended                178291
Canceled               6260
Postponed              2534
Abandoned                15
AP                       10
Retired                   8
Walkover                  7
AET                       4
Coverage canceled         1
Name: count, dtype: int64

In [25]:
df = df[df['status_description'].isin(status_description_set)]
df = df.sort_values('startTimestamp')
df = df.reset_index(drop=True)

In [26]:
df.shape

(178291, 19)

In [27]:
df.isna().sum()

winnerCode                     11
id                              0
startTimestamp                  0
tournament_slug                 0
tournament_category_slug        0
season_name                    15
status_description              0
status_type                     0
homeTeam_slug                   0
awayTeam_slug                   0
homeScore_period1             848
homeScore_period2           43006
homeScore_normaltime          469
awayScore_period1             848
awayScore_period2           43004
awayScore_normaltime          469
1                           14325
2                           14324
X                           14316
dtype: int64

# Missing data

## home/awayScore

In [28]:
df.loc[(df['homeScore_period1'].notna()) & (df['homeScore_period2'].notna()) & (df['homeScore_normaltime'].isna()), 'homeScore_normaltime'] = df['homeScore_period1'] + df['homeScore_period2']
df.loc[(df['homeScore_period1'].notna()) & (df['homeScore_normaltime'].notna()) & (df['homeScore_period2'].isna()), 'homeScore_period2'] = df['homeScore_normaltime'] - df['homeScore_period1']
df.loc[(df['homeScore_period2'].notna()) & (df['homeScore_normaltime'].notna()) & (df['homeScore_period1'].isna()), 'homeScore_period1'] = df['homeScore_normaltime'] - df['homeScore_period2']

In [29]:
df.loc[(df['awayScore_period1'].notna()) & (df['awayScore_period2'].notna()) & (df['awayScore_normaltime'].isna()), 'awayScore_normaltime'] = df['awayScore_period1'] + df['awayScore_period2']
df.loc[(df['awayScore_period1'].notna()) & (df['awayScore_normaltime'].notna()) & (df['awayScore_period2'].isna()), 'awayScore_period2'] = df['awayScore_normaltime'] - df['awayScore_period1']
df.loc[(df['awayScore_period2'].notna()) & (df['awayScore_normaltime'].notna()) & (df['awayScore_period1'].isna()), 'awayScore_period1'] = df['awayScore_normaltime'] - df['awayScore_period2']

In [30]:
df.isna().sum()

winnerCode                     11
id                              0
startTimestamp                  0
tournament_slug                 0
tournament_category_slug        0
season_name                    15
status_description              0
status_type                     0
homeTeam_slug                   0
awayTeam_slug                   0
homeScore_period1             840
homeScore_period2            1299
homeScore_normaltime          465
awayScore_period1             840
awayScore_period2            1299
awayScore_normaltime          465
1                           14325
2                           14324
X                           14316
dtype: int64

## season_name

In [31]:
for row in df.loc[df['season_name'].isna(), ['id', 'startTimestamp', 'tournament_category_slug', 'tournament_slug']].values:
    row_id, row_timestamp, row_country, row_tournament = row
    
    temp_df = df.loc[(df['season_name'].notna()) & (df['tournament_category_slug'] == row_country) & (df['tournament_slug'] == row_tournament)].sort_values('startTimestamp').reset_index(drop=True)
    date_index = abs(temp_df['startTimestamp'] - row_timestamp).sort_values().index[0]
    season_name_imput = temp_df.iloc[date_index]['season_name']

    df.loc[df['id'] == row_id, 'season_name'] = season_name_imput

## odds

In [32]:
split_len = df['1'].str.split('/').str.len()
split_len = split_len[split_len == 1].index
df.loc[split_len, '1'] = pd.NA

split_len = df['2'].str.split('/').str.len()
split_len = split_len[split_len == 1].index
df.loc[split_len, '2'] = pd.NA

split_len = df['X'].str.split('/').str.len()
split_len = split_len[split_len == 1].index
df.loc[split_len, 'X'] = pd.NA

calculate missing odds per tournament season

In [33]:
df_count = df.loc[:, ['tournament_category_slug', 'tournament_slug', 'season_name', 'id']]\
    .groupby(by=['tournament_category_slug', 'tournament_slug', 'season_name'], as_index=False).count().rename(columns={'id': 'size'})

df_1_null = df.loc[:, ['tournament_category_slug', 'tournament_slug', 'season_name', '1']]\
    .groupby(by=['tournament_category_slug', 'tournament_slug', 'season_name'], as_index=False).agg({'1': lambda x: x.isnull().sum()})

df_X_null = df.loc[:, ['tournament_category_slug', 'tournament_slug', 'season_name', 'X']]\
    .groupby(by=['tournament_category_slug', 'tournament_slug', 'season_name'], as_index=False).agg({'X': lambda x: x.isnull().sum()})

df_2_null = df.loc[:, ['tournament_category_slug', 'tournament_slug', 'season_name', '2']]\
    .groupby(by=['tournament_category_slug', 'tournament_slug', 'season_name'], as_index=False).agg({'2': lambda x: x.isnull().sum()})

In [34]:
df_odds = df_count.merge(df_1_null, on=['tournament_category_slug', 'tournament_slug', 'season_name'])\
    .merge(df_X_null, on=['tournament_category_slug', 'tournament_slug', 'season_name'])\
        .merge(df_2_null, on=['tournament_category_slug', 'tournament_slug', 'season_name'])

df_odds['1_%'] = round(df_odds['1'] / df_odds['size'], 2)
df_odds['X_%'] = round(df_odds['X'] / df_odds['size'], 2)
df_odds['2_%'] = round(df_odds['2'] / df_odds['size'], 2)

limit = 0.12
df_odds['remove'] = False
df_odds.loc[df_odds['1_%'].gt(limit) | df_odds['X_%'].gt(limit) | df_odds['2_%'].gt(limit), 'remove'] = True

In [36]:
df_odds.sample(25)

,tournament_category_slug,tournament_slug,season_name,size,1,X,2,1_%,X_%,2_%,remove
17,argentina,liga-profesional,Superliga 2021,325,1,1,2,0.00,0.00,0.01,False
428,serbia,mozzart-bet-superliga,Superliga 21/22,240,13,14,13,0.05,0.06,0.05,False
104,england,league-one,League One 20/21,552,0,0,0,0.00,0.00,0.00,False
292,italy,serie-c-girone-b,"Serie C, Group B 21/22",380,21,22,21,0.06,0.06,0.06,False
395,portugal,liga-portugal-betclic,Primeira Liga 15/16,306,0,0,0,0.00,0.00,0.00,False
71,czech-republic,fnl,FNL 15/16,210,0,0,0,0.00,0.00,0.00,False
306,japan,jleague,J.League 2018,306,0,0,0,0.00,0.00,0.00,False
526,turkey,tff-3-lig-grup-3,3rd Lig 17/18,303,303,303,303,1.00,1.00,1.00,True
78,czech-republic,fnl,FNL 22/23,240,8,8,8,0.03,0.03,0.03,False
383,poland,i-liga,I Liga 22/23,306,0,0,0,0.00,0.00,0.00,False


In [37]:
df_odds_remove = df_odds.loc[df_odds['remove'] == False]
odds_set = (df_odds_remove['tournament_category_slug'] + df_odds_remove['tournament_slug'] + df_odds_remove['season_name']).values

In [38]:
df = df.loc[(df['tournament_category_slug'] + df['tournament_slug'] + df['season_name']).isin(odds_set)]
df = df.sort_values('startTimestamp').reset_index(drop=True)

In [39]:
df.shape

(156097, 19)

## remaining missing data

In [40]:
df.isna().sum()

winnerCode                    11
id                             0
startTimestamp                 0
tournament_slug                0
tournament_category_slug       0
season_name                    0
status_description             0
status_type                    0
homeTeam_slug                  0
awayTeam_slug                  0
homeScore_period1            579
homeScore_period2            932
homeScore_normaltime         359
awayScore_period1            579
awayScore_period2            932
awayScore_normaltime         359
1                           1350
2                           1353
X                           1348
dtype: int64

# Save df to csv

In [41]:
df.to_csv(os.path.join(paths.INTERIM_DATA_DIR, 'events_with_odds_filtered.csv'), index=False)